In [6]:
import numpy as np
import pandas as pd
from scipy.signal import correlate
import matplotlib.pyplot as plt
from scipy.sparse import lil_matrix
from itertools import product
from itertools import combinations
from datetime import datetime, timedelta
from multiprocessing import Pool


# Load the temporal series data
data_with_date = pd.read_csv('temporal_series_18.csv')
data = data_with_date.drop(columns=['date'])

#Inizialize parameters
max_tau = 15 
threshold = 0.05

#Function to compute maximum of time correlation function between two temporal series
def time_delayed_cross_correlation(df, col1, col2, max_tau, threshold=0.05):
    """
    Optimized custom time-delayed cross-correlation function for two time series.

    Args:
    df (pandas.DataFrame): Dataframe containing the time series data.
    col1 (str): Name of the first time series column.
    col2 (str): Name of the second time series column.
    max_tau (int): Maximum time delay (positive or negative).
    threshold (float): Threshold for maximum correlation.

    Returns:
    float: Maximum cross-correlation above the threshold, or 0 if none.
    """
    # Extract the series and compute means and std deviations
    series_i = df[col1].values
    series_j = df[col2].values
    mean_i, std_i = series_i.mean(), series_i.std()
    mean_j, std_j = series_j.mean(), series_j.std()
    
    # Check if standard deviations are zero to avoid division by zero
    if std_i == 0 or std_j == 0:
        return 0
    
    # Initialize array to store cross-correlations for each lag
    cross_corrs = np.zeros(2 * max_tau + 1)

    # Compute cross-correlation for each lag
    for tau in range(-max_tau, max_tau + 1):
        if tau < 0:
            # Positive lag: shift series_j forward by -tau (series_i aligns with delayed series_j)
            numerator = np.sum((series_i[-tau:] - mean_i) * (series_j[:len(series_j) + tau] - mean_j))
            denominator = (std_i * std_j * (len(series_i) + tau))
        else:
            # Negative or zero lag: shift series_i forward by tau (series_j aligns with delayed series_i)
            numerator = np.sum((series_i[:len(series_i) - tau] - mean_i) * (series_j[tau:] - mean_j))
            denominator = (std_i * std_j * (len(series_i) - tau))

        # Calculate cross-correlation, check for zero denominator
        cross_corrs[tau + max_tau] = numerator / denominator if denominator != 0 else 0

    # Find the maximum correlation and compare with threshold
    max_corr = np.max(cross_corrs)
    max_index = np.argmax(cross_corrs)
    best_tau = max_index - max_tau  # Convert array index back to tau value

    # Apply threshold check and adjust max_corr based on tau direction
    if max_corr >= threshold:
        return max_corr if best_tau >= 0 else -max_corr
    else:
        return 0


#Function to compute the whole adjacency matrix (impossible to compute)
def ccr_matrix(df):
    #build the adjacency matrix
    adjacency_matrix = pd.DataFrame(np.nan, index=df.columns, columns=df.columns)

    #iterate for each pair of columns
    for col1 in df.columns:
        for col2 in df.columns:
            adjacency_matrix.loc[col1,col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)

    return adjacency_matrix

#function to compute just a submatrix of the adjacency matrix (submatricese on the diagonal)
def compute_submatrix(df, columns, filename, max_tau):
    submatrix = pd.DataFrame(index=columns, columns=columns)

    # Loop through column pairs, skipping redundant calculations
    for col1, col2 in product(columns, repeat=2):
        if col1 <= col2:  # Ensures each pair is calculated only once
            correlation = time_delayed_cross_correlation(df, col1, col2, max_tau)
            submatrix.loc[col1, col2] = correlation
            submatrix.loc[col2, col1] = correlation  # Fill symmetric position

    submatrix.to_csv(filename)
    print(f"Submatrix {filename} has been saved.")
    return submatrix

#function to computer a submatrix which is not on the diagonal
def compute_cross_group_matrix(df, group_a, group_b, filename):
    cross_matrix = pd.DataFrame(index=group_a, columns=group_b)
    for col1, col2 in product(group_a, group_b):
        cross_matrix.loc[col1, col2] = time_delayed_cross_correlation(df, col1, col2, max_tau)
    cross_matrix.to_csv(filename)
    print(f"submatrix {filename} has been saved")
    return cross_matrix

#Now to compute the submatrices we need to divide the 3104 columns into groups
def divide_into_groups(df, group_sizes):
    #Divides the comlumns into specified sizes
    columns = df.columns.tolist()
    groups = []
    start = 0
    for size in group_sizes:
        groups.append(columns[start:start + size])
        start += size
    return groups

#Change of program, now we will try to compute manually the single pieces
group_1 = data.columns[0:1000]
group_2 = data.columns[1000:2000]
group_3 = data.columns[2000:3104]


filename_sub_1 = "18_sub_mat_1.csv"
compute_submatrix(data, group_1, filename_sub_1, max_tau)

filename_sub_2 = "18_sub_mat_2.csv"
compute_submatrix(data, group_2, filename_sub_2, max_tau)

filename_sub_3 = "18_sub_mat_3.csv"
compute_submatrix(data, group_3, filename_sub_3, max_tau)

filename_cross_1 = "18_cross_matrix_1_2.csv"
compute_cross_group_matrix(data, group_1, group_2, filename_cross_1)

filename_cross_2 = "18_cross_matrix_1_3.csv"
compute_cross_group_matrix(data, group_1, group_3, filename_cross_2)

filename_cross_3 = "18_cross_matrix_2_3.csv"
compute_cross_group_matrix(data, group_2, group_3, filename_cross_3)


Submatrix 18_sub_mat_1.csv has been saved.
Submatrix 18_sub_mat_2.csv has been saved.
Submatrix 18_sub_mat_3.csv has been saved.
submatrix 18_cross_matrix_1_2.csv has been saved
submatrix 18_cross_matrix_1_3.csv has been saved
submatrix 18_cross_matrix_2_3.csv has been saved


,38099.0,38101.0,38103.0,38105.0,39001.0,39003.0,39005.0,39007.0,39009.0,39011.0,...,56027.0,56029.0,56031.0,56033.0,56035.0,56037.0,56039.0,56041.0,56043.0,56045.0
21083.0,0.233831,0.167984,0,0.292396,0.715086,0,0.528989,0,0.623451,0.108941,...,0.236867,0.558487,0.541382,0.499698,0.546115,0.667247,0.817185,0.704328,0.468756,0.279434
21085.0,0.261668,0.364959,0.222827,0.396304,0.611031,0.088414,0.419283,0,0.545452,0.228503,...,0.299244,0.710518,0.55381,0.516338,0.621602,0.754927,0.771483,0.676215,0.602975,0.407252
21087.0,0.136893,0.225245,0,0.310866,-0.506908,0,0.304597,0,-0.464956,0.164247,...,0.230811,0.38178,0.341056,0.33334,0.411428,-0.33235,-0.42967,0.510796,0.487565,0.210327
21089.0,0.346331,0.367673,0.251816,0.396152,0.554132,0.186555,0.440197,0,0.508637,0.25414,...,0.288792,0.706502,0.61619,0.556813,0.646739,0.805197,0.797467,0.627758,0.581178,0.355596
21091.0,0.273165,0.24468,0.149606,0.356607,0.591299,0.139136,0.400522,0,0.479529,0.204287,...,0.20518,0.612862,-0.515257,0.499865,0.566754,0.619251,0.66815,0.600865,0.464201,0.328017
...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...,...
38089.0,0.376422,0.718545,0.575818,0.665028,-0.560162,0.256775,-0.483136,0.231837,-0.447418,-0.417879,...,-0.413091,-0.676029,-0.469116,-0.634808,-0.737577,-0.466851,-0.371643,-0.573097,-0.762345,0.486189
38091.0,-0.387732,0.401395,0.391019,0.343289,-0.238953,-0.324954,-0.306495,0.309562,-0.264552,0.442621,...,-0.256557,-0.412386,-0.246385,-0.288304,-0.430717,-0.331187,-0.212243,-0.285624,0.408974,-0.266298
38093.0,0.47631,0.65636,0.501202,-0.622523,-0.3765,0.53522,-0.34678,0.403846,-0.295734,0.572344,...,-0.390612,-0.435502,-0.326246,-0.391487,-0.539692,-0.315976,-0.267464,-0.403253,-0.469655,-0.35621
38095.0,0.339379,0.343053,0.436018,-0.351758,0,-0.307516,-0.239362,-0.270273,0,-0.336006,...,-0.258706,-0.226111,-0.232494,-0.326241,-0.220237,0.174137,0.205318,0.159521,-0.318089,-0.397696
